In [1]:
import boto3 

from boto3.dynamodb.conditions import Key, Attr

import pandas as pd

In [2]:
dynamodb = boto3.resource( 'dynamodb' )

In [3]:
adspp_table = dynamodb.Table( 'ads_passenger_processed' )

In [4]:
scan_output = adspp_table.scan()

In [12]:
l = []

for item in scan_output[ 'Items' ]:

    l.append( item[ 'topic' ] )

df = pd.DataFrame()

df[ 'topics' ] = l

df[ 'topics' ].value_counts()

topics
/apollo/sensor/gnss/raw_data                        301
/apollo/sensor/gnss/corrected_imu                   174
/apollo/localization/msf_status                     165
/apollo/localization/pose                           160
/apollo/sensor/gnss/odometry                        157
/tf                                                 144
/apollo/canbus/chassis_detail                        63
/apollo/control                                      58
/apollo/canbus/chassis                               54
/apollo/sensor/velodyne/fusion/PointCloud2Status     16
/apollo/monitor                                       9
/apollo/perception/traffic_light                      7
/apollo/sensor/radar/front                            6
/apollo/sensor/gnss/gnss_status                       3
/apollo/routing_response_history                      2
/apollo/sensor/gnss/ins_stat                          2
/apollo/sensor/gnss/best_pose                         2
/apollo/hmi/status                       

In [15]:
query_input = dict()

query_output = adspp_table.scan( **query_input )

#

l = []

for item in query_output[ 'Items' ]:

    l.append( item[ 'topic' ] )

#

i = 1 

while ( 'LastEvaluatedKey' in query_output.keys() ) and ( i <= 10 ):

    query_input[ 'ExclusiveStartKey' ] = query_output[ 'LastEvaluatedKey' ]
    
    query_output = adspp_table.scan( **query_input )

    for item in query_output[ 'Items' ]:

        l.append( item[ 'topic' ] )

    i = i + 1

In [16]:
len( l )

13796

In [17]:
df = pd.DataFrame()

df[ 'topics' ] = l

df[ 'topics' ].value_counts()

topics
/apollo/sensor/gnss/raw_data                        3180
/apollo/sensor/gnss/odometry                        1702
/apollo/sensor/gnss/corrected_imu                   1699
/tf                                                 1687
/apollo/localization/pose                           1677
/apollo/localization/msf_status                     1666
/apollo/control                                      578
/apollo/canbus/chassis_detail                        571
/apollo/canbus/chassis                               564
/apollo/sensor/velodyne/fusion/PointCloud2Status     106
/apollo/sensor/radar/front                            94
/apollo/perception/traffic_light                      88
/apollo/monitor                                       64
/apollo/sensor/gnss/ins_stat                          36
/apollo/sensor/gnss/best_pose                         24
/apollo/sensor/gnss/gnss_status                       19
/apollo/routing_response_history                      16
/apollo/sensor/gnss/ins_

In [22]:
query_input = dict(
    
                   IndexName = 'topic-index',
    
                   Limit = 2000,
    
                   FilterExpression = Attr( 'groupMetadataID' ).eq( '1f70a4f0-f0e0-11ee-ba1e-fb353e7798cd' ),

                   KeyConditionExpression = Key( 'topic' ).eq( '/apollo/perception/obstacles' ),
    
                  )

query_output = adspp_table.query( **query_input )

#

query_output_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

#

i = 1 

while ( 'LastEvaluatedKey' in query_output.keys() ) and ( i <= 10 ):

    query_input[ 'ExclusiveStartKey' ] = query_output[ 'LastEvaluatedKey' ]
    
    query_output = adspp_table.query( **query_input )

    temp_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

    query_output_df = pd.concat( [ query_output_df, temp_df ] )

    i = i + 1

""
